<a href="https://colab.research.google.com/github/msabdulrazak/Datasets/blob/main/Experiment9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Setting the wikipedia Api


In [1]:
pip install wikipedia-api pydantic

  Preparing metadata (setup.py) ... done
ERROR: Operation cancelled by user

Extracting details of multinational companies from their wikipedia


In [2]:
import wikipediaapi
import re

class InstitutionDetails:
    def __init__(self, name, founded, headquarters, branches, summary):
        self.name = name
        self.founded = founded
        self.headquarters = headquarters
        self.branches = branches
        self.summary = summary

    def __str__(self):
        return f"""
Institution Details
-------------------
Name: {self.name}
Founded: {self.founded}
Headquarters: {self.headquarters}
Branches: {", ".join(self.branches) if self.branches else "Not Found"}

Summary:
{self.summary}
"""

def fetch_institution_info(institution_name):
    wiki = wikipediaapi.Wikipedia(user_agent="Mozilla/5.0", language="en")
    page = wiki.page(institution_name)

    if not page.exists():
        return f"No Wikipedia page found for {institution_name}"

    text = page.text

    # Extract Founded Year
    founded = extract_infobox(page).get("founded") or extract_infobox(page).get("established")
    if not founded:
        founded_match = re.search(r"(?:Founded|Established)(?: on| in)?:?\s*([\w\s,]+ \d{4})", text, re.IGNORECASE)
        founded = founded_match.group(1).strip() if founded_match else "Not Found"

    # Extract Headquarters
    headquarters = extract_infobox(page).get("headquarters")
    if not headquarters:
        headquarters_match = re.search(r"(?:Headquarters|Head office|Headquartered in):?\s*([\w\s,]+)", text, re.IGNORECASE)
        headquarters = headquarters_match.group(1).strip() if headquarters_match else "Not Found"

    # Extract Branches
    branch_keywords = ["New York", "San Francisco", "London", "Tokyo", "Bangalore", "Cambridge", "Seattle", "Mountain View"]
    branches_found = {match for match in branch_keywords if re.search(rf"\b{match}\b", text, re.IGNORECASE)}
    branches = list(branches_found) if branches_found else ["Not Found"]

    # Extract Summary
    summary_sentences = re.split(r'(?<=\.)\s', page.summary)[:3]  # Split into sentences and take first 3
    summary = " ".join(summary_sentences).strip()

    return InstitutionDetails(
        name=institution_name,
        founded=founded,
        headquarters=headquarters,
        branches=branches,
        summary=summary
    )

def extract_infobox(page):
    """Extract infobox details from a Wikipedia page (if available)."""
    infobox = {}
    for line in page.text.split("\n"):
        match = re.match(r"\|\s*([^=]+)\s*=\s*(.+)", line)
        if match:
            key, value = match.groups()
            key = key.strip().lower().replace(" ", "_")
            value = value.strip()
            infobox[key] = value
    return infobox

# Example Usage:
institution_name = input("Enter institution name: ")
result = fetch_institution_info(institution_name)
print(result)


ModuleNotFoundError: No module named 'wikipediaapi'